In [1]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [2]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)
import os
#Edit this to match your choice of main_path
main_path = "/content/drive/My Drive/11777_multimodal_course"
os.chdir(main_path)
#!pip --version
os.chdir("11777_VIST_project/data_reformatting/stories_jsons")

Mounted at /content/drive


In [3]:
#!git clone https://github.com/styfeng/11777_VIST_project.git
#stories_test.json  stories_train.json  stories_val.json

# Preprocess Stories and Captions into Lists

In [ ]:
import json

def process_stories_captions(json_file):
    f = open(json_file,'r')
    input_json = json.load(f)
    story_lst = []
    caption_lst = []
    for k,v in input_json.items():
        story_lst.append(v['sent_texts'])
        inner_captions_lst = []
        for inner_lst in v['captions']:
            inner_captions_lst.extend(inner_lst)
        caption_lst.append(inner_captions_lst)
    print(len(story_lst))
    print(len(caption_lst))
    return story_lst, caption_lst

In [ ]:
story_lst, caption_lst = process_stories_captions('stories_train.json')

9778
9778


In [ ]:
print(story_lst[:10])
print(caption_lst[:10])

[['our landmark tree in town was about to be destroyed and cleared for a new mall .', 'so we decided to take the day to go out and enjoy its beauty .', 'to see the final glimpse of the roots , extending out into the depths of the hill .', 'and its magnificent trunk , larger than life itself .', 'one last picture of its beauty so we could capture it forever .'], ['we found this tree when we were walking in a nearby town .', 'it turns out it is a popular attraction here .', 'the tree is very unusual , with its roots exposed .', 'the trunk was really wide , as much as 12 feet !', 'you can see how big these roots are - pretty amazing !'], ['pictures of a tree are taken .', 'the top of the tree is taken .', 'another part of the tree mostly the roots .', 'some more different parts of the tree .', 'and some more parts of the tree is taking .'], ['they went to the botanic gardens specifically to see the large tree .', 'there was an informational sign posted near it .', 'the roots were huge and

# Length and Diversity Metrics

In [ ]:
#functions for length, TTR, UTR

from string import punctuation
import re
import pkg_resources
import numpy as np
import nltk
nltk.download('punkt')
from nltk import word_tokenize


def get_length(document):
    word_lst = word_tokenize(document)
    return len(word_lst)


#get type-token ratio of document
def TTR_score(document):
    word_lst = word_tokenize(document)
    clean_word_lst = []

    for word in word_lst:
        clean_word_lst.append(word)

    unique_word_lst = set(clean_word_lst)
    if len(clean_word_lst) == 0:
        TTR = 0
    else:
        TTR = len(unique_word_lst) / len(clean_word_lst)
    #print("Document: ", document, " / TTR: ", TTR)
    return TTR


#get unique-trigram ratio of document
def UTR_score(document):
    # returns the unique trigram fraction in this population.
    # Higher the unique trigram fraction, more the diversity
    unique_trigrams = set()
    total_trigrams = 0

    #for i,hyp_i in enumerate(hyp_population):
    document_words = document.strip().split()
    if len(document_words)>=3:
        total_trigrams += len(document_words)-2
        for j in range(len(document_words)-2):
            trigram = " ".join(document_words[j:j+2])
            unique_trigrams.add(trigram)

    unique_trigram_fraction = len(unique_trigrams)/(total_trigrams+1e-10)
    if total_trigrams == 0: unique_trigram_fraction = 0.0
    return unique_trigram_fraction

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
import time
from tqdm.auto import tqdm, trange

#get length, TTR, and UTR of stories
story_char_len_lst = []
story_len_lst = []
story_TTR_lst = []
story_UTR_lst = []

for inner_stories in tqdm(story_lst):
    story_char_len_lst.append(np.average([len(doc) for doc in inner_stories]))
    story_len_lst.append(np.average([get_length(doc) for doc in inner_stories]))
    story_TTR_lst.append(np.average([TTR_score(doc) for doc in inner_stories]))
    story_UTR_lst.append(np.average([UTR_score(doc) for doc in inner_stories]))

print(len(story_char_len_lst))
print(len(story_len_lst))
print(len(story_TTR_lst))
print(len(story_UTR_lst))
 
print("Average results for stories: {} char len, {} word len, {} TTR, {} UTR".format(np.average(story_char_len_lst),np.average(story_len_lst),np.average(story_TTR_lst),np.average(story_UTR_lst)))
#Average results for stories: 54.81863366741665 char len, 11.742687666189404 word len, 0.9560179945592294 TTR, 0.997368194292269 UTR

  0%|          | 0/9778 [00:00<?, ?it/s]

9778
9778
9778
9778
Average results for stories: 54.81863366741665 char len, 11.742687666189404 word len, 0.9560179945592294 TTR, 0.997368194292269 UTR


In [ ]:
#get length, TTR, and UTR of captions
caption_char_len_lst = []
caption_len_lst = []
caption_TTR_lst = []
caption_UTR_lst = []

for inner_captions in tqdm(caption_lst):
    if len(inner_captions) > 0:
        caption_char_len_lst.append(np.average([len(doc) for doc in inner_captions]))
        caption_len_lst.append(np.average([get_length(doc) for doc in inner_captions]))
        caption_TTR_lst.append(np.average([TTR_score(doc) for doc in inner_captions]))
        caption_UTR_lst.append(np.average([UTR_score(doc) for doc in inner_captions]))  

print(len(caption_char_len_lst))
print(len(caption_len_lst))
print(len(caption_TTR_lst))
print(len(caption_UTR_lst))
 
print("Average results for captions: {} char len, {} len, {} TTR, {} UTR".format(np.average(caption_char_len_lst),np.average(caption_len_lst),np.average(caption_TTR_lst),np.average(caption_UTR_lst)))
#57.80595066671349 char len, 11.931376508739273 len, 0.9286765023666008 TTR, 0.996854476091794 UTR

  0%|          | 0/9778 [00:00<?, ?it/s]

9491
9491
9491
9491
Average results for captions: 57.80595066671349 char len, 11.931376508739273 len, 0.9286765023666008 TTR, 0.996854476091794 UTR


In [ ]:
# save individual diversity results to files

story_diversity_dict = {}
story_diversity_dict["char_length"] = story_char_len_lst
story_diversity_dict["word_length"] = story_len_lst
story_diversity_dict["TTR"] = story_TTR_lst
story_diversity_dict["UTR"] = story_UTR_lst

caption_diversity_dict = {}
caption_diversity_dict["char_length"] = caption_char_len_lst
caption_diversity_dict["word_length"] = caption_len_lst
caption_diversity_dict["TTR"] = caption_TTR_lst
caption_diversity_dict["UTR"] = caption_UTR_lst

In [ ]:
out_story_diversity_file = 'story_diversity_results.json'
out_caption_diversity_file = 'caption_diversity_results.json'

import json
with open(os.path.join(main_path, out_story_diversity_file),"w") as diversity_f:
    json.dump(story_diversity_dict,diversity_f,indent=4)
diversity_f.close()
print("Story diversity results written to file")

import json
with open(os.path.join(main_path, out_caption_diversity_file),"w") as diversity_f:
    json.dump(caption_diversity_dict,diversity_f,indent=4)
diversity_f.close()
print("Caption diversity results written to file")

Story diversity results written to file
Caption diversity results written to file


# Most Frequent Words

In [ ]:
story_word_dict = {}
for inner_stories in tqdm(story_lst):
    for doc in inner_stories:
        word_lst = word_tokenize(doc)
        for word in word_lst:
            if word not in story_word_dict.keys():
                story_word_dict[word] = 0
            else:
                story_word_dict[word] += 1

counter = 0
for w in sorted(story_word_dict, key=story_word_dict.get, reverse=True):
    if counter < 200:
        print(w, story_word_dict[w])
    counter += 1
#print(sorted(story_word_dict, key=story_word_dict.get, reverse=True)[:100])
#['.', 'the', 'a', 'to', 'and', 'was', 'of', 'we', ',', 'i', 'it', 'in', 'for', 'on', 'they', 'is', 'were', 'this', '[', ']', 'at', 'with', 'had', 'my', '!', 'there', 'all', 'some', 'he', 'that', 'out', 'so', 'day', 'male', 'up', 'our', 'time', "'s", 'her', 'his', 'are', 'went', 'as', 'got', 'she', 'great', 'their', 'beach', 'female', 'but', 'location', 'one', 'see', 'family', 'people', 'fun', 'friends', 'beautiful', 'took', 'from', 'night', 'be', "n't", 'everyone', 'very', 'after', 'get', 'by', 'birthday', 'here', 'party', 'go', 'have', 'many', 'even', 'like', 'picture', 'an', 'water', 'decided', 'then', 'could', 'little', 'when', 'around', 'cake', 'made', 'good', 'me', 'together', 'just', 'first', 'take', 'down', 'us', 'fireworks', 'really', 'off', 'today', 'lot']

  0%|          | 0/9778 [00:00<?, ?it/s]

. 46438
the 39992
a 17621
to 16625
and 11220
was 10999
of 9644
we 9334
, 8735
i 6528
it 5915
in 5758
for 5504
on 4995
they 4849
is 4701
were 4483
this 4277
[ 3969
] 3968
at 3907
with 3764
had 3664
my 3495
! 3284
there 3219
all 2834
some 2806
he 2743
that 2713
out 2545
so 2491
day 2447
male 2419
up 2326
our 2269
time 2242
's 2176
her 2122
his 2114
are 2047
went 2031
as 1997
got 1829
she 1802
great 1784
their 1712
beach 1688
female 1573
but 1563
location 1562
one 1504
see 1452
family 1448
people 1447
fun 1424
friends 1407
beautiful 1396
took 1396
from 1394
night 1365
be 1360
n't 1346
everyone 1333
very 1332
after 1321
get 1298
by 1263
birthday 1244
here 1220
party 1187
go 1139
have 1138
many 1108
even 1086
like 1073
picture 1065
an 1055
water 1041
decided 1018
then 1016
could 979
little 974
when 952
around 922
cake 910
made 903
good 901
me 900
together 895
just 894
first 888
take 875
down 868
us 866
fireworks 857
really 855
off 844
today 842
lot 814
saw 813
trip 793
you 787
what 785
them

In [ ]:
caption_word_dict = {}
for inner_captions in tqdm(caption_lst):
    for doc in inner_captions:
        word_lst = word_tokenize(doc)
        for word in word_lst:
            if word not in caption_word_dict.keys():
                caption_word_dict[word] = 0
            else:
                caption_word_dict[word] += 1

counter = 0
for w in sorted(caption_word_dict, key=caption_word_dict.get, reverse=True):
    if counter < 200:
        print(w, caption_word_dict[w])
    counter += 1
print(sorted(caption_word_dict, key=caption_word_dict.get, reverse=True)[:100])
#['a', '.', 'the', 'of', 'in', 'on', 'and', 'with', 'is', 'man', 'are', 'people', 'at', 'to', 'woman', 'two', 'an', ',', 'group', 'that', 'for', 'water', 'large', 'beach', 'sitting', 'standing', 'up', 'young', '[', ']', 'white', 'front', 'his', 'sky', 'blue', 'holding', 'table', 'picture', 'wearing', 'red', 'it', 'by', 'night', 'building', 'her', 'men', 'small', 'from', 'girl', 'down', 'over', 'looking', 'while', 'black', 'smiling', 'next', 'around', 'has', 'fireworks', 'cake', 'boy', 'as', 'out', 'their', 'near', 'together', 'women', 'three', 'ocean', 'outside', 'camera', 'sign', 'location', 'some', 'posing', 'green', 'day', 'city', 'playing', 'street', 'trees', 'walking', 'view', 'other', 'birthday', 'one', 'this', 'person', 'photo', 'organization', 'top', 'little', 'room', 'into', 'sits', 'couple', 'yellow', "'s", 'park', 'shirt']

  0%|          | 0/9778 [00:00<?, ?it/s]

a 145491
. 88036
the 70382
of 42622
in 41422
on 30086
and 27896
with 26398
is 24866
man 16104
are 15214
people 14781
at 14761
to 11665
woman 10978
two 9575
an 9265
, 9018
group 7738
that 7523
for 7499
water 6627
large 6621
beach 6571
sitting 6329
standing 6327
up 5936
young 5463
[ 5319
] 5318
white 5123
front 5118
his 4613
sky 4447
blue 4446
holding 4412
table 4367
picture 4215
wearing 4069
red 4013
it 3983
by 3914
night 3784
building 3779
her 3710
men 3697
small 3535
from 3534
girl 3347
down 3186
over 3183
looking 3152
while 3064
black 3047
smiling 2955
next 2949
around 2948
has 2918
fireworks 2913
cake 2906
boy 2899
as 2888
out 2869
their 2850
near 2849
together 2840
women 2832
three 2775
ocean 2764
outside 2740
camera 2683
sign 2674
location 2498
some 2494
posing 2490
green 2469
day 2398
city 2390
playing 2344
street 2341
trees 2333
walking 2308
view 2237
other 2155
birthday 2153
one 2146
this 2114
person 2103
photo 2071
organization 2037
top 2007
little 1957
room 1920
into 1898
sit

# POS Tagging

In [ ]:
import nltk
nltk.download('punkt')
from nltk import word_tokenize
import spacy
from collections import OrderedDict
#Note: run pip install spacy (v. 2.2.4) and python -m spacy download en_core_web_sm
nlp = spacy.load("en_core_web_sm") 
complete_POS_lst = ['POS', 'PUNCT', 'SYM', 'ADJ', 'CCONJ', 'NUM', 'DET', 'ADV', 'ADP', 'X', 'VERB', 'NOUN', 'PROPN', 'PART', 'INTJ', 'SPACE', 'PRON','AUX','CONJ','SCONJ','LITERAL','entities']


def get_POS(continuation):
    POS_dict = OrderedDict()
    for key in complete_POS_lst:
        POS_dict[key] = [0,0]
    words = continuation.split()
    for x in words:
        for char in x:
            if char.isnumeric():
                POS_dict['LITERAL'][0] += 1
                POS_dict['LITERAL'][1] += 1
                break
    tokenized_cont = nlp(continuation)
    total_len = len(tokenized_cont)
    for ent in tokenized_cont.ents:
        POS_dict['entities'][0] += 1
        POS_dict['entities'][1] += len(ent)
    for token in tokenized_cont:
        if token.pos_ in POS_dict.keys():
            POS_dict[token.pos_][0] += 1
            POS_dict[token.pos_][1] += 1
        if token.tag_ in POS_dict.keys():
            POS_dict[token.tag_][0] += 1
            POS_dict[token.tag_][1] += 1            
    for key, value in POS_dict.items():
        POS_dict[key][1] = value[1]/total_len
    return POS_dict    


def evaluate_POS(continuations):
    all_results = []
    for continuation in tqdm(continuations):
        POS = [get_POS(c) for c in continuation]
        if len(POS) != 0:
            result = OrderedDict()
            for key in complete_POS_lst:
                key_sum_0 = sum(c[key][0] for c in POS)
                key_sum_1 = sum(c[key][1] for c in POS)
                result[key] = [key_sum_0/len(POS), key_sum_1/len(POS)]
            all_results.append(result)
    
    final_results = OrderedDict()
    for key in complete_POS_lst:
        key_sum_0 = sum(c[key][0] for c in all_results)
        key_sum_1 = sum(c[key][1] for c in all_results)
        final_results[key] = [key_sum_0/len(all_results), key_sum_1/len(all_results)]         
    
    print(final_results)
    return final_results

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
story_POS_results = evaluate_POS(story_lst)
print(story_POS_results)
#OrderedDict([('POS', [0.024892616076907046, 0.0018361421649407514]), ('PUNCT', [1.4016363264471314, 0.12181968260194846]), ('SYM', [0.0009204336265084887, 6.761584424839115e-05]), ('ADJ', [0.7780527715279186, 0.0683075835434682]), ('CCONJ', [0.2683984454898766, 0.021274778366075428]), ('NUM', [0.044446717120064205, 0.0035933672978646475]), ('DET', [1.7303743096747781, 0.14768930920916742]), ('ADV', [0.5936387809367835, 0.050414793868112956]), ('ADP', [1.0849253426058538, 0.08841114513249568]), ('X', [0.0007568009817958686, 6.401081458517532e-05]), ('VERB', [1.3885048067089372, 0.11648092361746912]), ('NOUN', [2.3875843730824333, 0.2064910466333303]), ('PROPN', [0.11939046839844948, 0.010456323263542896]), ('PART', [0.3145223972182424, 0.023908317189457396]), ('INTJ', [0.010595213745142211, 0.0008137559920410851]), ('SPACE', [0.0, 0.0]), ('PRON', [0.8507465739415007, 0.07160067345914017]), ('AUX', [0.6814072407445322, 0.059865368752653435]), ('CONJ', [0.0, 0.0]), ('SCONJ', [0.12129269789323553, 0.00875781484842826]), ('LITERAL', [0.014031499284107248, 0.001152001649698791]), ('entities', [0.2446921660871402, 0.036152988834874285])])
#less nouns in stories, more verbs

  0%|          | 0/9778 [00:00<?, ?it/s]

OrderedDict([('POS', [0.024892616076907046, 0.0018361421649407514]), ('PUNCT', [1.4016363264471314, 0.12181968260194846]), ('SYM', [0.0009204336265084887, 6.761584424839115e-05]), ('ADJ', [0.7780527715279186, 0.0683075835434682]), ('CCONJ', [0.2683984454898766, 0.021274778366075428]), ('NUM', [0.044446717120064205, 0.0035933672978646475]), ('DET', [1.7303743096747781, 0.14768930920916742]), ('ADV', [0.5936387809367835, 0.050414793868112956]), ('ADP', [1.0849253426058538, 0.08841114513249568]), ('X', [0.0007568009817958686, 6.401081458517532e-05]), ('VERB', [1.3885048067089372, 0.11648092361746912]), ('NOUN', [2.3875843730824333, 0.2064910466333303]), ('PROPN', [0.11939046839844948, 0.010456323263542896]), ('PART', [0.3145223972182424, 0.023908317189457396]), ('INTJ', [0.010595213745142211, 0.0008137559920410851]), ('SPACE', [0.0, 0.0]), ('PRON', [0.8507465739415007, 0.07160067345914017]), ('AUX', [0.6814072407445322, 0.059865368752653435]), ('CONJ', [0.0, 0.0]), ('SCONJ', [0.1212926978

In [ ]:
caption_POS_results = evaluate_POS(caption_lst)
print(caption_POS_results)
#OrderedDict([('POS', [0.013067350355307351, 0.0009956983409839788]), ('PUNCT', [0.9513943033751485, 0.07553418832048796]), ('SYM', [0.001308842295039745, 7.677436699088921e-05]), ('ADJ', [0.930701600346531, 0.0785590709816394]), ('CCONJ', [0.24744670389492318, 0.019561201573895987]), ('NUM', [0.14812278298739448, 0.012562789244992848]), ('DET', [2.158621618141168, 0.18055039851134713]), ('ADV', [0.1909346866622251, 0.016280059623855148]), ('ADP', [1.7946797550896105, 0.14929805577932936]), ('X', [0.0009026094896919876, 5.936834497002133e-05]), ('VERB', [1.1365135391423409, 0.09649251228635021]), ('NOUN', [3.6065120172326783, 0.30418386794760993]), ('PROPN', [0.1136860651611509, 0.009779535414908351]), ('PART', [0.06198386775775845, 0.0048254819408441235]), ('INTJ', [0.0023531064517261923, 0.00020890710895237578]), ('SPACE', [0.0, 0.0]), ('PRON', [0.11265350800173823, 0.008564060578425486]), ('AUX', [0.42235158454209604, 0.03617752820441561]), ('CONJ', [0.0, 0.0]), ('SCONJ', [0.09262283566888174, 0.007315671770036081]), ('LITERAL', [0.01298013322562899, 0.0010339681578942197]), ('entities', [0.278661656071839, 0.03189411346334912])])

  0%|          | 0/9778 [00:00<?, ?it/s]

OrderedDict([('POS', [0.013067350355307351, 0.0009956983409839788]), ('PUNCT', [0.9513943033751485, 0.07553418832048796]), ('SYM', [0.001308842295039745, 7.677436699088921e-05]), ('ADJ', [0.930701600346531, 0.0785590709816394]), ('CCONJ', [0.24744670389492318, 0.019561201573895987]), ('NUM', [0.14812278298739448, 0.012562789244992848]), ('DET', [2.158621618141168, 0.18055039851134713]), ('ADV', [0.1909346866622251, 0.016280059623855148]), ('ADP', [1.7946797550896105, 0.14929805577932936]), ('X', [0.0009026094896919876, 5.936834497002133e-05]), ('VERB', [1.1365135391423409, 0.09649251228635021]), ('NOUN', [3.6065120172326783, 0.30418386794760993]), ('PROPN', [0.1136860651611509, 0.009779535414908351]), ('PART', [0.06198386775775845, 0.0048254819408441235]), ('INTJ', [0.0023531064517261923, 0.00020890710895237578]), ('SPACE', [0.0, 0.0]), ('PRON', [0.11265350800173823, 0.008564060578425486]), ('AUX', [0.42235158454209604, 0.03617752820441561]), ('CONJ', [0.0, 0.0]), ('SCONJ', [0.09262283

#Keywords

In [ ]:
# Install RAKE for keyword extraction
!pip3 install python-rake
import RAKE
Rake = RAKE.Rake("stopwords.txt")

In [ ]:
def extract_keywords(prompt):
    keyword_lst = []
    #format: Rake.run(prompt, minCharacters = X, maxWords = Y, minFrequency = Z)
    keywords = Rake.run(prompt)
    #print(keywords)
    for pair in keywords:
        words = pair[0].split()
        for word in words:
            keyword_lst.append(word)
    return keyword_lst

prompt = "first thing we do , let's fight all the lawyers"
#prompt = "i hate this restaurant . the food is horrible !"
keyword_lst = extract_keywords(prompt)
#print(keywords_dict)

In [ ]:
story_keywords_dict = {}
for inner_stories in tqdm(story_lst):
    for story in inner_stories:
        keyword_lst = extract_keywords(story)
        for word in keyword_lst:
            if word not in story_keywords_dict.keys():
                story_keywords_dict[word] = 0
            else:
                story_keywords_dict[word] += 1

counter = 0
print(sorted(story_keywords_dict, key=story_keywords_dict.get, reverse=True)[:100])  
for w in sorted(story_keywords_dict, key=story_keywords_dict.get, reverse=True):
    if counter < 200:
        print(w, story_keywords_dict[w])
    counter += 1  

  0%|          | 0/9778 [00:00<?, ?it/s]

['day', '[male]', 'time', 'beach', 'location', '[female]', 'family', 'people', 'fun', 'friends', 'beautiful', 'night', 'birthday', 'party', 'picture', 'water', 'decided', 'little', 'cake', 'fireworks', 'lot', 'trip', 'ride', 'city', 'kids', 'ready', 'view', 'happy', 'looked', 'race', 'look', 'food', 'park', 'started', 'home', 'finally', 'amazing', 'nice', 'friend', 'organization', 'pictures', 'excited', 'enjoyed', 'getting', 'favorite', 'found', 'vacation', 'town', 'loved', 'love', 'sun', 'looking', 'dinner', 'game', 'sky', 'set', 'walk', 'play', 'mom', 'building', 'house', 'photo', 'boat', 'ocean', 'pretty', 'bike', 'local', 'taking', 'outside', 'wheel', 'dog', 'cool', 'walked', 'eat', 'looks', 'girl', 'playing', 'played', 'arrived', 'couple', 'watch', 'dad', 'lights', 'start', 'enjoying', 'lots', 'baby', 'gathered', 'lake', 'celebrate', 'brought', 'walking', 'restaurant', 'enjoy', 'near', 'visit', 'car', 'inside', 'ferris', 'hotel']
day 2425
[male] 2320
time 2223
beach 1686
location 

In [ ]:
caption_keywords_dict = {}
for inner_captions in tqdm(caption_lst):
    for caption in inner_captions:
        keyword_lst = extract_keywords(caption)
        for word in keyword_lst:
            if word not in caption_keywords_dict.keys():
                caption_keywords_dict[word] = 0
            else:
                caption_keywords_dict[word] += 1

counter = 0
print(sorted(caption_keywords_dict, key=caption_keywords_dict.get, reverse=True)[:100])  
for w in sorted(caption_keywords_dict, key=caption_keywords_dict.get, reverse=True):
    if counter < 200:
        print(w, caption_keywords_dict[w])
    counter += 1  

  0%|          | 0/9778 [00:00<?, ?it/s]

['people', 'woman', 'water', 'beach', 'sitting', 'standing', 'front', 'white', 'sky', 'blue', 'holding', 'table', 'picture', 'red', 'wearing', 'night', 'building', 'girl', 'looking', 'black', 'smiling', 'fireworks', 'cake', 'boy', 'women', 'near', 'ocean', 'outside', 'camera', 'sign', 'posing', 'green', 'day', 'city', 'trees', 'street', 'playing', 'walking', 'view', '[location]', 'birthday', 'person', 'photo', 'top', 'little', '[organization]', 'couple', 'sits', 'shirt', 'yellow', 'park', 'lit', 'food', 'children', 'lights', 'background', 'dog', 'stands', 'child', 'riding', 'body', 'party', 'image', 'ride', 'dark', 'crowd', 'road', 'dressed', 'sand', 'sun', 'light', 'wheel', 'car', 'baby', 'snow', 'stand', 'sit', 'field', 'colorful', 'grass', 'taking', 'beautiful', 'boat', 'wall', 'display', 'pink', 'bike', 'family', 'tree', 'girls', 'pose', 'glasses', 'hat', 'buildings', 'tall', 'stage', 'restaurant', 'ferris', 'covered', 'friends']
people 14707
woman 10805
water 6594
beach 6525
sitti

# Calculate Statistical Significance P-Values

In [ ]:
# modified : cell inserted to calculate p-value (paired two-tailed t-test) of individual ROUGE/BERTScore/etc. results between generated text of original and perturbed inputs

import scipy
from scipy import stats
import numpy as np
import json

# load individual metrics values from two files: first for original generations, second for perturbation generations
def calculate_pvalues_from_jsons(orig_fn,perturb_fn):
    with open(orig_fn) as orig_f:
        orig_json = json.loads(orig_f.read())
    with open(perturb_fn) as perturb_f:
        perturb_json = json.loads(perturb_f.read())
    # calculate p-values (from paired two-tailed t-tests) for each key (e.g. ROUGE-2 and ROUGE-L):
    avg_dict = {}
    sig_dict = {}
    for key in orig_json.keys():
        #print(key)
        #print(orig_json[key])
        #print(perturb_json[key],'\n')
        avg_dict[key] = (np.average(orig_json[key]),np.average(perturb_json[key])) #get average metrics per key (for sanity check - should be approximately the same as results we already have), note: only approximately due to rounding/etc.
        sig_dict[key] = scipy.stats.ttest_rel(orig_json[key],perturb_json[key])
    return avg_dict, sig_dict

orig_fn = 'caption_diversity_results.json' #CHANGE appropriately (file with original scores)
perturb_fn = 'story_diversity_results.json' #CHANGE appropriately (file with perturbed scores)
#avg_dict, sig_dict = calculate_pvalues_from_jsons(orig_fn,perturb_fn)
#print(avg_dict)
#print(sig_dict)

#Example output (want pvalues to be below 0.05 for significance)
#{'length': (12.576157292185167, 9.777501244400199), 'TTR': (0.9457185788044364, 0.95752620015624), 'UTR': (0.9982761786587853, 0.9998340799573436)}
#{'length': Ttest_relResult(statistic=44.18127038233726, pvalue=0.0), 'TTR': Ttest_relResult(statistic=-9.661362129784136, pvalue=7.590963100695351e-22), 'UTR': Ttest_relResult(statistic=-7.283290524775888, pvalue=3.9006360471671226e-13)}

In [ ]:
# for .txt instead of .json files (e.g. perplexity)
def calculate_pvalues_from_txt(orig_fn,perturb_fn):
    with open(orig_fn) as orig_f:
        orig_json = [float(x.strip()) for x in orig_f.readlines()]
    with open(perturb_fn) as perturb_f:
        perturb_json = [float(x.strip()) for x in perturb_f.readlines()]
    # calculate p-values (from paired two-tailed t-tests) for each key (e.g. ROUGE-2 and ROUGE-L):
    avg_dict = {}
    sig_dict = {}
    avg_dict['PPL'] = (np.average(orig_json),np.average(perturb_json)) #get average metrics per key (for sanity check - should be approximately the same as results we already have), note: only approximately due to rounding/etc.
    sig_dict['PPL'] = scipy.stats.ttest_rel(orig_json,perturb_json)
    return avg_dict, sig_dict

#orig_fn = 'stat_sig/baseline/val_generated_steven_colab_cp5797_txt.txt_PPL_individual' #CHANGE appropriately (file with original scores)
#perturb_fn = 'stat_sig/chai/val_generated_chai_beam_11.txt_PPL_individual' #CHANGE appropriately (file with perturbed scores)
#avg_dict, sig_dict = calculate_pvalues_from_txt(orig_fn,perturb_fn)
#print(avg_dict)
#print(sig_dict)